In [109]:
import os
from PyPDF2 import PdfFileReader,PdfFileWriter
from docx import Document
import re
import logging
import pandas as pd
# from pyresparser import ResumeParser
# import nltk
# nltk.download()
# nltk.download('stopwords')

# from nltk.corpus import stopwords

# set(stopwords.words("english"))

In [110]:
folder_path="D:\IneuronInternship\Ineuron Batch\Resumes"

In [3]:
class WrongFileFormat(Exception):
    """User defined Exception for wrong file format
    Attributes:
        file -- file which caused the error
        message -- explanation of the error
    """
    
    def __init__(self,file_name,message):
        self.file_name=file_name
        self.message=message
        super().__init__(self.message)
    
    def __str__(self):
        return f'{self.file_name} -> {self.message}'
        

In [5]:
class InvalidFormat(Exception):
    """User defined Exception for invalid file format
    Attributes:
        file -- file which caused the error
        message -- explanation of the error
    """
    
    def __init__(self,file_name,message):
        self.file_name=file_name
        self.message=message
        super().__init__(self.message)
    
    def __str__(self):
        return f'{self.file_name} -> {self.message}'
        

In [6]:
class WrongFileName(Exception):
    """User defined Exception for wrong file name 
    Attributes:
        file_name -- file which caused the error
        message -- explanation of the error
    """
    
    def __init__(self,file_name,message):
        self.file_name=file_name
        self.message=message
        super().__init__(self.message)
    
    def __str__(self):
        return f'{self.file_name} -> {self.message}'
        

In [47]:
class searchFiles:
    """
    Files Searcher:gives files in directory
    Attributes:
        
        path -- path where files are present
    methods:
            startLogger(),getFilesInDirectory(),fileNames()
    returns:
    
    """
    def __init__(self,path):
        logging.info("Search Initialized....")
        self.folder_path=path
        self.total_files=[]
        self.pdf_list=[]
        self.docx_list=[]
        self.all_files=[]
        
    def startLogger(self):
        """
            starts logging

                    
        """
        logging.info("taking user input for file name")
        self.file_name=str(input("Enter File name for logs"))
        try:
            if type(self.file_name)== str:
                logging.info("file name given")
                if '.' in self.file_name:
                    raise WrongFileName(self.file_name,"pass only name of the file to be created")
                else:
                    logging.info("proper extension added for filename")
                    self.file_name=self.file_name+'.log'
                    logging.basicConfig(filename=self.file_name, encoding='utf-8', level=logging.DEBUG,format ='%(asctime)s %(levelname)s %(message)s')
                    print("File created")
                    logging.info("Log file created")
            else:
                
                raise WrongFileFormat(self.file_name,"Invalid format")
        except WrongFileFormat as wff:
            logging.info("wrong file format added exception raised")
        except WrongFileName as wfn:
            logging.info("wrong file extension added exception raised")
                
                
            
        
        
    def getFilesInDirectory(self):
        logging.info("returning files in directory")
        list_of_files=os.listdir(folder_path)
        for files in list_of_files:
            self.total_files.append(files)
            
        for file in self.total_files:
            
            file_split=file.split('.')
            if 'docx' in file_split:
                self.all_files.append(file)
                self.docx_list.append(file)
            elif 'pdf' in file_split:
                self.all_files.append(file)
                self.pdf_list.append(file)
            else:
                logging.error("End of files reached")
        logging.info("files returned")
        return "Total files in path "+self.folder_path+" are: "+str(len(self.total_files))+" Pdf files are :"+str(len(self.pdf_list))+" and Doc files are :"+ str(len(self.docx_list))
    
    def fileNames(self):
        logging.info("asking for selection of file format")
        file_type=input("Enter file type you want to see.!!")
        logging.info("returning files in directory")
        try:
            if type(file_type)==str:
                if file_type=='pdf':
                    logging.info("selected file format : pdf")
                    if len(self.pdf_list)>0:
                        logging.info("returning pdf files")
                        return self.pdf_list
                    else:
                        logging.warning("no pdf files in this directory")
                        return "No files for file type"+file_type
                elif file_type=='docx' or file_type =='docs':
                    logging.info("selected format is  : docx")
                    if len(self.docx_list)>0:
                        logging.info("returning docs files")
                        return self.docx_list
                    else:
                        logging.warning("no docs files in this directory")
                        return "No files for file type"+file_type
                elif file_type=="both" or file_type=="all":
                    logging.info("returning all files")
                    return self.all_files
                                        
                else:
                    logging.error("different format file exists in directory")
                    raise InvalidFormat(str(file_type),"Make sure only pdf or docs or docx are in the directory")
            else:
                raise WrongFileFormat(str(file_type),"format mis-match")
        except InvalidFormat as inf:
            logging.error("different format file exists in directory")
        except WrongFileFormat as wff:
            logging.error("wrong file name passed other than string")
            
            
            

            

In [112]:
class ResumeSearcher():
    """
    Resume Searcher for github,linkedin,email
    Attributes:
        file_names -- list of file names 
        ###Note:list of file names can be obtained from filenames in "searchFiles" class
        folder_path -- path where resumes are present
    methods:
            getLinekedInData(),getEmail(),getGithubData()
    returns:
            list type data
    """
    def __init__(self,file_names,folder_path):
        logging.info("Resumer searcher intialized..")
        self.file_names=file_names
        self.folder_path=folder_path
        self.linkedin=[]
        self.github=[]
        self.email=[]
        self.skillset=[]
        self.allfiles=[]
        if len(file_names)==17:
            for f in file_names:
                self.allfiles.append(f)
        
        
    
    def getLinkedInData(self):
        """returns Linkedin id if present in docs or pdf"""
        logging.info("called for linkedin data")
        linkedin_pattern="www.linkedin.com/in/[a-zA-Z-0-9_]*"
        logging.info("LinkdeIn pattern intialized..")
        for doc_name in self.file_names:
            if 'docx' in doc_name.split('.'):
                logging.info("Parsing text in docs/docx")
                document=Document(doc_name)
                for para in document.paragraphs:
                    if len(para.text)>0:
                        linkedin_text=re.findall(linkedin_pattern,para.text)
                        if linkedin_text!="":
                            logging.info("data added to list from docs/docx")
                            self.linkedin.extend(linkedin_text)
                        else:
                            logging.warning("no search found ind docs/docx")
                            print("no match found")
                    else:
                        logging.warning("End of file")
                        print("End of File")
                            #print(document.paragraphs[2].text)
            elif 'pdf' in doc_name.split('.'):
                logging.info("Parsing text in pdf")
                file = open(doc_name, 'rb')
                pdfreader=PdfFileReader(f)
                for page_num in range(pdfreader.numPages):
                    pageObj=pdfreader.getPage(page_num)
                    linkedin_text=re.findall(linkedin_pattern,pageObj.extractText())
                    if linkedin_text!="":
                        logging.info("data added to list from pdf")
                        self.linkedin.extend(linkedin_text)
                    else:
                        logging.warning("no search found in  pdf")
            else:
                logging.error("no search found for pdf and docs/docx")
                raise InvalidFormat(doc_name,"Make sure only pdf or docs or docx are in the directory")
                
        return self.linkedin
            
    
    def getEmail(self):
        """returns email id if present in docs or pdf"""
        logging.info("called for email data")
        email_pattern="[^!@#$%^&*()]*@[gmail.com]+"
        logging.info("email pattern intialized..")
        for doc_name in self.file_names:
            
            if 'docx' in doc_name.split('.'):
                logging.info("Parsing text in docs/docx")
                document=Document(doc_name)
                for para in document.paragraphs:
                    
                    if len(para.text)>0:
                        email_text=re.findall(email_pattern,para.text)
                        
                        if email_text!='':
                            logging.info("data added to list from docs/docx")
                            self.email.extend(email_text)
                        else:
                            logging.warning("no search found ind docs/docx")
                    else:
                        logging.warning("End of file")
                        print("End of File")
                            #print(document.paragraphs[2].text)
            elif 'pdf' in doc_name.split('.'):
                logging.info("Parsing text in pdf")
                file = open(doc_name, 'rb')
                pdfreader=PdfFileReader(file)
                for page_num in range(pdfreader.numPages):
                    pageObj=pdfreader.getPage(page_num)
                    email_text=re.findall(email_pattern,pageObj.extractText())
                    if email_text!="":
                        logging.info("data added to list from pdf")
                        self.email.extend(email_text)
                    else:
                        logging.warning("no search found in  pdf")
            else:
                logging.error("no search found for pdf and docs/docx")
                raise InvalidFormat(doc_name,"Make sure only pdf or docs or docx are in the directory")
                
        return self.email
    
    def getGithubData(self):
        """returns github id if present in docs or pdf"""
        logging.info("called for github data")
        github_pattern="github.com/[a-zA-Z0-9]+"
        logging.info("github pattern intialized..")
        for doc_name in self.file_names:
            
            if 'docx' in doc_name.split('.'):
                logging.info("Parsing text in docs/docx")
                document=Document(doc_name)
                for para in document.paragraphs:
                    if len(para.text)>0:
                        github_text=re.findall(github_pattern,para.text)
                        if github_text!='':
                            logging.info("data added to list from docs/docx")
                            self.github.append(github_text)
                        else:
                            logging.warning("no search found ind docs/docx")
                            print("no match found")
                    else:
                        logging.warning("End of file")
                        print("End of File")
                            #print(document.paragraphs[2].text)
            elif 'pdf' in doc_name.split('.'):
                logging.info("Parsing text in pdf")
                file = open(doc_name, 'rb')
                pdfreader=PdfFileReader(file)
                for page_num in range(pdfreader.numPages):
                    pageObj=pdfreader.getPage(page_num)
                    github_text=re.findall(github_pattern,pageObj.extractText())
                    if github_text!="":
                        logging.info("data added to list from pdf")
                        self.github.extend(github_text)
                    else:
                        logging.warning("no search found in  pdf")
                        print("no match found")
            else:
                logging.error("no search found for pdf and docs/docx")
                raise InvalidFormat(doc_name,"Make sure only pdf or docs or docx are in the directory")                
        return self.github
    
    def getSkillsSet(self):
        pass
    
    def getAllFiles(self):
        return self.allfiles
            

In [113]:
search_files=searchFiles(folder_path)

In [114]:
search_files.startLogger()

Enter File name for logsineuron
File created


In [115]:
search_files.getFilesInDirectory()

ERROR:root:End of files reached
ERROR:root:End of files reached
ERROR:root:End of files reached


'Total files in path D:\\IneuronInternship\\Ineuron Batch\\Resumes are: 20 Pdf files are :11 and Doc files are :6'

In [116]:
pdf_files=search_files.fileNames()

Enter file type you want to see.!!pdf


In [117]:
pdf_files

['15+ (1).pdf',
 '15+.pdf',
 '20.pdf',
 '3+.pdf',
 '5+ .pdf',
 '5+.pdf',
 '5.pdf',
 '6+.pdf',
 '7+.pdf',
 'freasher .pdf',
 'mteh fresher.pdf']

In [94]:
all_files=search_files.fileNames()

Enter file type you want to see.!!both


In [95]:
all_files

['12+ (2).docx',
 '12+.docx',
 '15+ (1).pdf',
 '15+.pdf',
 '20.pdf',
 '3+ (2).docx',
 '3+.docx',
 '3+.pdf',
 '4+.docx',
 '5+ .pdf',
 '5+.pdf',
 '5.pdf',
 '6+.pdf',
 '7+.pdf',
 '8+.docx',
 'freasher .pdf',
 'mteh fresher.pdf']

In [96]:
docs_file_name=search_files.fileNames()

Enter file type you want to see.!!docx


In [97]:
docs_file_name

['12+ (2).docx', '12+.docx', '3+ (2).docx', '3+.docx', '4+.docx', '8+.docx']

In [118]:
resume_searcher1=ResumeSearcher(docs_file_name,folder_path)
resume_searcher2=ResumeSearcher(all_files,folder_path)
resume_searcher3=ResumeSearcher(pdf_files,folder_path)

In [119]:
resume_searcher1.getAllFiles()

[]

In [120]:
resume_searcher2.getAllFiles()

['12+ (2).docx',
 '12+.docx',
 '15+ (1).pdf',
 '15+.pdf',
 '20.pdf',
 '3+ (2).docx',
 '3+.docx',
 '3+.pdf',
 '4+.docx',
 '5+ .pdf',
 '5+.pdf',
 '5.pdf',
 '6+.pdf',
 '7+.pdf',
 '8+.docx',
 'freasher .pdf',
 'mteh fresher.pdf']

In [101]:
resume_searcher3.getAllFiles()

[]

In [121]:
linked_data=resume_searcher.getLinkedInData()

In [122]:
linked_data

[]

In [58]:
email_data=resume_searcher.getEmail()

In [59]:
email_data

[]

In [60]:
github_data=resume_searcher.getGithubData()

In [107]:
files_names=resume_searcher2.file_names

In [131]:
col_email=[]
col_linkedin=[]
col_git=[]

In [132]:
for i in email_data:
    col_email.append(i)

In [135]:
for i in linked_data:
    col_linkedin.append(i)

In [134]:
for i in github_data:
    col_git.append(i)

In [136]:
d = {'email_id':col_email, 'github_id':col_git, 'linkedin_id':col_linkedin}

In [137]:
d

{'email_id': [], 'github_id': [], 'linkedin_id': []}

In [ ]:
df = pd.DataFrame(d, index=files_names)

In [139]:
df

,email_id,github_id,linkedin_id
12+ (2).docx,[],[],[]
12+.docx,[],[],[]
15+ (1).pdf,[],[],[]
15+.pdf,[],[],[]
20.pdf,[],[],[]
3+ (2).docx,[],[],[]
3+.docx,[],[],[]
3+.pdf,[],[],[]
4+.docx,[],[],[]
5+ .pdf,[],[],[]
